# 제4 고지 : 신경망 만들기 
## STEP 40 : 브로드캐스트 함수

이전 단계에서 `sum` 함수를 구현했는데, 이 함수의 역전파에서는 구현되어 있지 않던 `broadcast_to`함수를 이용했다.  
이번 단계에서는 해당 함수를 구현하도록 한다. 또한 DeZero에서 넘파이와 같은 브로드캐스트를 할 수 있도록 한다.

### 40.1 broadcast_to 함수와 sum_to 함수(넘파이 버전)

먼저 이해를 돕기 위해 `np.broadcast_to(x,shape)` 함수를 살펴보겠다. 이 함수는 `ndarray` 인스턴스인 `x`를 복제하여 `shape` 인수로 지정한 형상이 되도록 한다. 

```python
import numpy as np 
x = np.array([1,2,3])
y = np.broadcast_to(x,(2,3))
print(y)
'''
[[1 2 3]
 [1 2 3]]
'''
```

위의 결과와 같이 해당 함수는 `(3,)` -> `(2,3)` 형상으로 변환하는데, 이때 **1차원 배열의 원소를 복사**했다.  
그렇다면 **브로드캐스트 후의 역전파**는 어떻게 될까? 

<p align='center'>
    <img src='../assets/deep_learning_2_images/fig%201-20.png' align='center' width='50%'>
    <img src='../assets/deep_learning_2_images/fig%201-21.png' align='center' width='50%'>
</p>

위의 그림을 통해 이해해보면, `y = x+x` 와 같이 **분기노드의 경우 역전파는 단순히 합해서 계산**한다. 이를 **$N$ 개의 분기로 확장**해서 생각해볼 수 있고, **브로드캐스팅 처럼 원소가 복사 되는 경우 기울기를 합하면 된다**는 것을 알 수 있다.

즉, `np.broadcast_to` 같은 함수는 다음과 같이  작동한다.
<p align='center'>
    <img src='../assets/%EA%B7%B8%EB%A6%BC%2040-1.png' align='center' width='50%'>
</p>

여기서 우리가 구현해야 할 것은 입력의 형상과 같아지도록 기울기의 합을 구하는 `sum_to(x,shape)` 함수만 있다면 해결할 수 있다. 그러나 넘파이에는 해당 함수가 존재하지 않으므로 `dezero/utils.py`에 넘파이 버전 `sum_to` 를 구현한다.

```python
def sum_to(x, shape):
    """Sum elements along axes to output an array of a given shape.

    Args:
        x (ndarray): Input array.
        shape:

    Returns:
        ndarray: Output array of the shape.
    """
    ndim = len(shape)
    lead = x.ndim - ndim
    lead_axis = tuple(range(lead))

    axis = tuple([i + lead for i, sx in enumerate(shape) if sx == 1])
    y = x.sum(lead_axis + axis, keepdims=True)
    if lead > 0:
        y = y.squeeze(lead_axis)
    return y
```

해당 함수를 이용하면 다음과 같은 계산이 가능하다.
```python
import numpy as np 
from dezero.utils import sum_to 

x = np.array([[1,2,3],[4,5,6]])
y = sum_to(x,(1,3))
print(y)
print("="*10)
y = sum_to(x,(2,1))
print(y)
'''
[[5 7 9]]
==========
[[ 6]
 [15]]
'''
```

이 함수는 **`np.sum()` 함수와 기능은 같지만, 인수를 주는 방법이 다르다.**

그렇다면 이어서 `sum_to` 함수의 역전파는 어떻게 구성할까? 명백하게도 이는 `broadcast_to` 함수를 이용하여 구할 수 있다.

<p align='center'>
    <img src='../assets/%EA%B7%B8%EB%A6%BC%2040-2.png' align='center' width='50%'>
</p>

### 40.2 broadcast_to 함수와 sum_to 함수(DeZero 버전)

```python
class BroadcastTo(Function):
    def __init__(self, shape):
        self.shape = shape

    def forward(self, x):
        self.x_shape = x.shape
        y = np.broadcast_to(x, self.shape)
        return y

    def backward(self, gy):
        ##############################
        gx = sum_to(gy, self.x_shape)
        ##############################
        return gx


def broadcast_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return BroadcastTo(shape)(x)

class SumTo(Function):
    def __init__(self, shape):
        self.shape = shape

    def forward(self, x):
        self.x_shape = x.shape
        y = utils.sum_to(x, self.shape)
        return y

    def backward(self, gy):
        ##############################
        gx = broadcast_to(gy, self.x_shape)
        ##############################
        return gx


def sum_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return SumTo(shape)(x)

```




### 40.3 브로드캐스트 대응

이번 단계에서 `sum_to` 함수를 구현한 이유는 바로 넘파이 브로드캐스트에 대응하기 위함이였다.  
앞서 언급했듯이, 브로드캐스트란 다른 형상의 다차원 배열끼리의 연산을 다음과 같이 가능하게 하는데, 

<p align='center'>
    <img src='../assets/deep_learning_2_images/fig%201-3.png' align='center' width='50%'>
    <img src='../assets/deep_learning_2_images/fig%201-4.png' align='center' width='50%'>
</p>

현재 Dezero는 순전파의 경우 `ndarray` 로 구현했기 때문에 브로드캐스트에 대응 가능하지만 **역전파의 경우 브로드캐스트의 역전파가 일어나지 않는다.**  
그래서 이를 반영하기 위해 다음과 같이 `Add` 클래스를 수정한다. 여기서 주목해야할 것은 `broadcast_to` 함수의 역전파는 `sum_to` 함수에 해당한다는 것이다.

```python
class Add(Function):
    def forward(self, x0, x1):
        ##################################################
        self.x0_shape, self.x1_shape = x0.shape, x1.shape
        ##################################################
        y = x0 + x1
        return y

    def backward(self, gy):
        gx0, gx1 = gy, gy
        ##################################################
        if self.x0_shape != self.x1_shape:  # for broadcaset
            gx0 = dezero.functions.sum_to(gx0, self.x0_shape)
            gx1 = dezero.functions.sum_to(gx1, self.x1_shape)
        ##################################################
        return gx0, gx1

```

In [2]:
import sys
sys.path.append("..")

import numpy as np 
from dezero import Variable

x0 = Variable(np.array([1,2,3]))
x1 = Variable(np.array([10]))
y = x0+x1 
print(f"순전파에서의 broadcast : {y}") 

y.backward()
print(f"역전파에서의 broadcast : {x1.grad}")

순전파에서의 broadcast : variable([11 12 13])
역전파에서의 broadcast : variable([3])
